<a href="https://colab.research.google.com/github/daisysong76/AI--Machine--learning/blob/main/Data_for_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating Data for Retrieval Augmented Generation.
Generating data for Retrieval Augmented Generation (RAG) is essential because RAG models combine the strengths of both retrieval-based and generative models to enhance their performance in generating more accurate, contextually relevant, and information-rich responses. Here are some reasons why generating data for RAG is crucial:

RAG models can retrieve more relevant documents or information snippets. This enriches the model's contextual understanding.

Training on Diverse Data Sources: Generating comprehensive and diverse datasets allows these models to refine their retrieval mechanisms, ensuring that the most relevant and useful information is retrieved to aid in the generation process.

Addressing Data Scarcity in Specific Domains

In this notebook, we'll generate a dataset for RAG using `sciq` dataset from Huggingface.

In [3]:
!pip install llama-index==0.9.48 datasets tqdm python-dotenv spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.9 MB/s eta 0:00:00


## Setup Environment Variables

In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

## Download sciq dataset

Note: Parts of this repo are inspired by [this](https://srk.ai/blog/004-ai-llm-retrieval-eval-llamaindex) blog on evaluating the performance of RAGs.

In [4]:
from datasets import load_dataset

dataset = load_dataset("sciq")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Generate RAG Dataset From the Documents

In [5]:
# Let's see what's in the dataset.

dataset["train"].to_pandas().head()

,question,distractor3,distractor1,distractor2,correct_answer,support
0,What type of organism is commonly used in prep...,viruses,protozoa,gymnosperms,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,tropical effect,muon effect,centrifugal effect,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,endothermic,unbalanced,reactive,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,zeta decay,beta decay,gamma decay,alpha decay,All radioactive decay is dangerous to living t...
4,Kilauea in hawaii is the world’s most continuo...,magma,greenhouse gases,carbon and smog,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...


We have a `question`, and `support` fields that are super relevant. The answer is also present in the `correct_answer` field, but is one worded.

The cleanup_cols function takes a single argument, row, which is expected to be a dictionary-like object containing keys for "question", "correct_answer", and "support". The function constructs and returns a new dictionary with three key-value pairs.

In [6]:
def cleanup_cols(row):
    return {"question": row["question"], "answer": row["correct_answer"].title(), "support": row["support"]}

updated_dataset = dataset.map(cleanup_cols)

Map:   0%|          | 0/11679 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
updated_dataset = updated_dataset.remove_columns(["distractor3", "distractor1", "distractor2", "correct_answer"])
updated_dataset["train"].to_pandas().head()

,question,support,answer
0,What type of organism is commonly used in prep...,"Mesophiles grow best in moderate temperature, ...",Mesophilic Organisms
1,What phenomenon makes global winds blow northe...,Without Coriolis Effect the global winds would...,Coriolis Effect
2,Changes from a less-ordered state to a more-or...,Summary Changes of state are examples of phase...,Exothermic
3,What is the least dangerous radioactive decay?,All radioactive decay is dangerous to living t...,Alpha Decay
4,Kilauea in hawaii is the world’s most continuo...,Example 3.5 Calculating Projectile Motion: Hot...,Smoke And Ash


### Save the Dataset in HuggingFace format.

In [8]:
import os

content_folder = os.path.join(os.path.abspath(""), ".content")
print(f"Dataset contains {len(updated_dataset)} rows")
updated_dataset.save_to_disk(content_folder + "/rag_sciq_data.hf")

Dataset contains 3 rows


Saving the dataset (0/1 shards):   0%|          | 0/11679 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Great! Now we're ready to build our RAG.

In [9]:
updated_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'support', 'answer'],
        num_rows: 11679
    })
    validation: Dataset({
        features: ['question', 'support', 'answer'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'support', 'answer'],
        num_rows: 1000
    })
})